In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mixtral-8x7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="cuda")

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= '1' # 

In [3]:
from datasets import load_dataset
eval_dataset = load_dataset('json', data_files='testing.json', split='train')
eval_dataset

Dataset({
    features: ['Findings', 'Medical Condition'],
    num_rows: 3612
})

In [5]:
eval_dataset['Medical Condition'][0]

'70 year old man s/p AVR REASON FOR THIS EXAMINATION:  eval for ischemic event/ emboli No contraindications for IV contrast\n\nBRAIN VOLUMETRIC DATA:\nGrey Matter: 565.45\nWhite matter: 446.24\ncsf: 1.42\nTotal Brain Volume: 1011.69\nTotal Intracranial Volume: 1602.74\nTotal Ventricular Volume: 36.66\nLateral Ventricles Volume: 34.05\nFirst Ventricle Volume: 17.12\nSecond Ventricle Volume: 16.93\nThird Ventricle Volume: 1.26\nFourth Ventricle Volume: 1.36'

In [8]:
eval_prompt = f"Given medical condition: {eval_dataset['Medical Condition'][0]}. Generate a radiological findings section and add in volumetric data where applicable. ### Sample Output: "
print(eval_prompt)

Given medical condition: 70 year old man s/p AVR REASON FOR THIS EXAMINATION:  eval for ischemic event/ emboli No contraindications for IV contrast

BRAIN VOLUMETRIC DATA:
Grey Matter: 565.45
White matter: 446.24
csf: 1.42
Total Brain Volume: 1011.69
Total Intracranial Volume: 1602.74
Total Ventricular Volume: 36.66
Lateral Ventricles Volume: 34.05
First Ventricle Volume: 17.12
Second Ventricle Volume: 16.93
Third Ventricle Volume: 1.26
Fourth Ventricle Volume: 1.36. Generate a radiological findings section and add in volumetric data where applicable. ### Sample Output: 


In [9]:
# Re-init the tokenizer so it doesn't add padding or eos token
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=150, repetition_penalty=1.15)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Given medical condition: 70 year old man s/p AVR REASON FOR THIS EXAMINATION:  eval for ischemic event/ emboli No contraindications for IV contrast

BRAIN VOLUMETRIC DATA:
Grey Matter: 565.45
White matter: 446.24
csf: 1.42
Total Brain Volume: 1011.69
Total Intracranial Volume: 1602.74
Total Ventricular Volume: 36.66
Lateral Ventricles Volume: 34.05
First Ventricle Volume: 17.12
Second Ventricle Volume: 16.93
Third Ventricle Volume: 1.26
Fourth Ventricle Volume: 1.36. Generate a radiological findings section and add in volumetric data where applicable. ### Sample Output: ----------------------------------------------- Cerebral Hemispheres The cerebral hemispheres are normal size, shape and position bilaterally with no evidence of mass effect or shift. There is mild prominence of the sulci at this time consistent with age related volume loss.   Basal Ganglia The basal ganglia appear unremarkable on both sides without any focal lesions identified. Midline structures The midline appears to

In [10]:
import json
generated_texts = {}
for idx, example in enumerate(eval_dataset):
    eval_prompt = f"Generate a findings section based on the medical condition: {example['Medical Condition']} and add in volumetric data where applicable. ### Sample output:"
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    model.eval()
    with torch.no_grad():
        generated_texts[f"{idx}"] = tokenizer.decode(model.generate(**model_input, max_new_tokens=150, repetition_penalty=1.15)[0], skip_special_tokens=True)
    print(f"Completed generation for example {idx}")
    if idx == 149:  
        break

with open("generated_texts2.json", "w") as f:
    json.dump(generated_texts, f)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Completed generation for example 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Completed generation for example 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Completed generation for example 146


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Completed generation for example 147


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Completed generation for example 148
Completed generation for example 149


In [12]:
import json
import re
from nltk.translate.bleu_score import corpus_bleu

with open("generated_texts2.json", "r") as f:
    generated_texts = json.load(f)


generated_summaries = []
for text in generated_texts.values():
    match = re.search(r'#+\s*(.*)', text)
    if match:
        generated_summaries.append(match.group(1).strip())
    else:
        generated_summaries.append(text) 
    


generated_summaries = [s.replace("Sample output: ", "") for s in generated_summaries]

print("Generated Summaries:", generated_summaries)


Generated Summaries: ['The brain volume was measured at ________. This value represents an increase of ___% compared to normal values, which are typically between __and__. A larger than expected size may be due to hydrocephalus or other causes such as tumor growth that compresses surrounding structures (e.g., meningiomas). Smaller-than average measurements could indicate cerebral atrophy from Alzheimer’s disease; however it can also occur with aging because there has been less time available since birth when neurons were created during development so they have had more opportunity over their lifetimes before dying off completely through natural processes like apoptosis.”', 'The patient has an acute subarachnoid hemorrhage (SAH) involving both sylvian fissures as well as the interhemispheric fissure. There is also evidence of intraventricular blood within all four ventricles. This SAH appears to be secondary to ruptured anterior communicating artery aneurysm which was confirmed by CTA. 

In [13]:
generated_tokenized = [text.split() for text in generated_summaries]
generated_tokenized

[['The',
  'brain',
  'volume',
  'was',
  'measured',
  'at',
  '________.',
  'This',
  'value',
  'represents',
  'an',
  'increase',
  'of',
  '___%',
  'compared',
  'to',
  'normal',
  'values,',
  'which',
  'are',
  'typically',
  'between',
  '__and__.',
  'A',
  'larger',
  'than',
  'expected',
  'size',
  'may',
  'be',
  'due',
  'to',
  'hydrocephalus',
  'or',
  'other',
  'causes',
  'such',
  'as',
  'tumor',
  'growth',
  'that',
  'compresses',
  'surrounding',
  'structures',
  '(e.g.,',
  'meningiomas).',
  'Smaller-than',
  'average',
  'measurements',
  'could',
  'indicate',
  'cerebral',
  'atrophy',
  'from',
  'Alzheimer’s',
  'disease;',
  'however',
  'it',
  'can',
  'also',
  'occur',
  'with',
  'aging',
  'because',
  'there',
  'has',
  'been',
  'less',
  'time',
  'available',
  'since',
  'birth',
  'when',
  'neurons',
  'were',
  'created',
  'during',
  'development',
  'so',
  'they',
  'have',
  'had',
  'more',
  'opportunity',
  'over',
  'th

In [14]:
with open("testing.json", "r") as f:
    testing_data = json.load(f)

reference_summaries = [example['Findings'] for example in testing_data[:150]]

reference_tokenized = [text.split() for text in reference_summaries]
reference_tokenized

[['No',
  'acute',
  'intracranial',
  'hemorrhage,',
  'large',
  'vascular',
  'territory',
  'infarct,',
  'edema',
  'or',
  'mass',
  'effect',
  'is',
  'present.',
  'The',
  'ventricles',
  'and',
  'sulci',
  'are',
  'normal',
  'in',
  'size',
  'and',
  'configuration.',
  'Chronic',
  'lacunar',
  'infarcts',
  'are',
  'noted',
  'in',
  'the',
  'right',
  'putamen',
  'and',
  'left',
  'subinsular',
  'white',
  'matter',
  '(446.24).',
  'There',
  'are',
  'mucus',
  'retention',
  'cysts',
  'in',
  'both',
  'maxillary',
  'sinuses',
  'as',
  'well',
  'as',
  'air-fluid',
  'levels',
  'in',
  'the',
  'frontal,',
  'ethmoidal',
  'and',
  'sphenoid',
  'sinuses,',
  'likely',
  'related',
  'to',
  'endotracheal',
  'intubation.'],
 ['There',
  'is',
  'a',
  'hyperdensity',
  'along',
  'the',
  'convexity',
  'of',
  'the',
  'right',
  'frontal,',
  'parietal,',
  'and',
  'temporal',
  'lobes',
  'with',
  'some',
  'areas',
  'of',
  'hypodensity',
  'that'

In [15]:
bleu_score = corpus_bleu([[ref] for ref in reference_tokenized], generated_tokenized)
bleu_score

0.012855908128278108

In [16]:
from rouge_score import rouge_scorer

In [18]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
total_scores = {metric: 0 for metric in ['rouge1', 'rouge2', 'rougeL']}
num_pairs = len(generated_summaries)
for gen_summary, ref_summary in zip(generated_summaries, reference_summaries):

    scores = scorer.score(gen_summary, ref_summary)
    for metric in scores:
        total_scores[metric] += scores[metric].fmeasure

average_scores = {metric: total_scores[metric] / num_pairs for metric in total_scores}

for metric in average_scores:
    print(f'Average {metric} score: {average_scores[metric]}')

Average rouge1 score: 0.1815269582533662
Average rouge2 score: 0.024939621874060264
Average rougeL score: 0.10400946358394735


In [19]:
from bert_score import score
P, R, F1 = score(generated_summaries, reference_summaries, lang='en', verbose=True)

print('Precision:', P.mean())
print('Recall:', R.mean())
print('F1 Score:', F1.mean())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/5 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 4.16 seconds, 36.08 sentences/sec
Precision: tensor(0.8284)
Recall: tensor(0.8203)
F1 Score: tensor(0.8241)
